In [1]:
import os
from os import listdir
from os.path import isfile, join
import sys
import pandas as pd

# probs_path = '/home/ubuntu/data/d03_classification/probabilities/'
probs_path = '../probabilities/'
sys.path.append(probs_path)

In [2]:
text_files = [f for f in listdir(probs_path) if isfile(join(probs_path,f))]
print(text_files)

## TODO: make sure we get study / image info. Join accordingly for ground truth

['view_23_e5_class_11-Mar-2018_dcm_sample_labelled_probabilities.txt', 'view_23_e5_class_11-Mar-2018_dicomsample_probabilities.txt']


In [108]:
f = text_files[0]

infile = open(probs_path + f)
infile = infile.readlines()
infile = [i.rstrip() for i in infile]
infile = [i.split('\t') for i in infile]

In [109]:
max_vals = []
max_inds = []
# if probability < 0.01, set to 0
for i in range(len(infile)):
    for j in range(len(infile[0])):
        val = infile[i][j]
        #print(val)
        if i!=0 and j!=0 and j!=1:
            if float(val) < 0.01:
                infile[i][j] = 0.0
    if i!=0:
        row_probs = list(map(float, infile[i][2:]))
        max_vals.append(max(row_probs))
        max_inds.append(row_probs.index(max(row_probs)))
# print(max_vals)
print(max_inds)

[13, 14, 5, 16, 16, 16, 13, 8, 16, 16, 16, 2, 16, 8, 21, 16, 16, 21, 16, 1, 1, 16, 1, 3, 16, 16, 4, 7, 1, 16, 16, 16, 16, 16, 16, 20, 16, 16, 16, 18, 16, 8, 16, 14, 14, 13, 14, 14, 1, 13, 16, 14, 16, 14, 1, 21, 16, 1, 20, 16, 16, 16, 16, 8, 1, 16, 16, 16, 3, 16, 14, 8, 8, 2, 1, 16, 1, 16, 21, 16, 1, 8, 1, 14, 16, 16, 16, 16, 16, 1, 19, 16, 14, 16, 16, 16, 11, 13, 16, 16, 16, 8, 16, 16, 14, 5, 14, 16, 16, 8, 8, 5, 1, 1, 18, 16, 16, 16, 16, 16, 16, 16, 16, 7, 16, 1, 16, 16, 1, 1, 1, 21, 16, 7]


In [48]:
df = pd.DataFrame(infile)
# df = pd.DataFrame(all_probs_list)

df.columns = infile[0]
df.drop(0, inplace=True)
df.drop(labels='study', axis=1, inplace=True)
print(df.shape)

df.head()

(134, 24)


,image,plax_far,plax_plax,plax_laz,psax_az,psax_mv,psax_pap,a2c_lvocc_s,a2c_laocc,a2c,...,a4c_laocc,a4c,a5c,other,rvinf,psax_avz,suprasternal,subcostal,plax_lac,psax_apex
1,a_90353_E8R9KEOU.dcm,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00020000001,0.0,...,0.9973,0.0018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,a_111698_1NCF3RYV.dcm,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.97349995,0.0244,0.00040000002,0.0,0.0,0.0,0.0,0.0,0.0
3,a_91046_EE9Q02L9.dcm,0.030699998,0.0006,0.0007000001,0.00040000002,0.0174,0.30920002,0.0006,0.0172,0.0006,...,0.0113,0.000100000005,0.0,0.22569999,0.0492,0.0007999999,0.017,0.1007,0.015700001,0.1911
4,a_114916_55OETDIM.dcm,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5,a_124700_D2I30O65.dcm,0.0,0.0,0.0,0.0,0.0,0.000100000005,0.0,0.0,0.0,...,0.0,0.0,0.0,0.9992,0.0,0.0,0.0,0.0,0.00020000001,0.0


In [50]:
# parse long string name into study, instance
fn_list = list(df['image'].tolist())
study_list = []
inst_list = []
for fn in fn_list:
    sub = fn.split('_')
    study_list.append(sub[1])
    subsub = sub[2].split('.')
    inst_list.append(subsub[0])

df['study'] = study_list
df['instancefilename'] = inst_list

# rearrange columns, drop image col
cols = df.columns.tolist()
cols_new = cols[-2:] + cols[1:-2]

df_new = df[cols_new]

print(len(df_new['instancefilename'].unique().tolist()))
print(df_new.shape)
df_new.head()

134
(134, 25)


,study,instancefilename,plax_far,plax_plax,plax_laz,psax_az,psax_mv,psax_pap,a2c_lvocc_s,a2c_laocc,...,a4c_laocc,a4c,a5c,other,rvinf,psax_avz,suprasternal,subcostal,plax_lac,psax_apex
1,90353,E8R9KEOU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00020000001,...,0.9973,0.0018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,111698,1NCF3RYV,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.97349995,0.0244,0.00040000002,0.0,0.0,0.0,0.0,0.0,0.0
3,91046,EE9Q02L9,0.030699998,0.0006,0.0007000001,0.00040000002,0.0174,0.30920002,0.0006,0.0172,...,0.0113,0.000100000005,0.0,0.22569999,0.0492,0.0007999999,0.017,0.1007,0.015700001,0.1911
4,114916,55OETDIM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5,124700,D2I30O65,0.0,0.0,0.0,0.0,0.0,0.000100000005,0.0,0.0,...,0.0,0.0,0.0,0.9992,0.0,0.0,0.0,0.0,0.00020000001,0.0


In [111]:
cols_view = df_new.columns.tolist()[2:]
# print(cols)

In [115]:
view_pred = []
for ind in max_inds:
    view_pred.append(cols_view[ind])
print(len(view_pred))

134


In [119]:
df_new['view_pred'] = view_pred
df_new.head()

,study,instancefilename,plax_far,plax_plax,plax_laz,psax_az,psax_mv,psax_pap,a2c_lvocc_s,a2c_laocc,...,a4c,a5c,other,rvinf,psax_avz,suprasternal,subcostal,plax_lac,psax_apex,view_pred
1,90353,E8R9KEOU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00020000001,...,0.0018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,a4c_laocc
2,111698,1NCF3RYV,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.97349995,0.0244,0.00040000002,0.0,0.0,0.0,0.0,0.0,0.0,a4c
3,91046,EE9Q02L9,0.030699998,0.0006,0.0007000001,0.00040000002,0.0174,0.30920002,0.0006,0.0172,...,0.000100000005,0.0,0.22569999,0.0492,0.0007999999,0.017,0.1007,0.015700001,0.1911,psax_pap
4,114916,55OETDIM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,other
5,124700,D2I30O65,0.0,0.0,0.0,0.0,0.0,0.000100000005,0.0,0.0,...,0.0,0.0,0.9992,0.0,0.0,0.0,0.0,0.00020000001,0.0,other


In [120]:
# Add views to join
sys.path.append('../../dicom_image_classification/src/d00_utils/')
from db_utils import dbReadWriteData

In [121]:
io_views = dbReadWriteData()
io_views.schema = 'views'
labels_df = io_views.get_table('instances_with_labels')
labels_df.head()

,sopinstanceuid,instancefilename,instanceidk,view,studyidk
0,1.2.840.113663.1500.1.365900332.3.3.20120703.9...,8Z0BWWL5,696108,plax,39531
1,1.2.840.113663.1500.1.365900332.3.4.20120703.9...,8Z0BWX0M,696109,plax,39531
2,1.2.840.113663.1500.1.365900332.3.7.20120703.9...,8Z0BWZGH,696112,a4c,39531
3,1.2.840.113663.1500.1.365900332.3.1.20120703.9...,8Z0BXKCB,696140,plax,39533
4,1.2.840.113663.1500.1.365900332.3.14.20120703....,8Z0BXYUW,696157,plax,39533


In [123]:
merge_df = df_new.merge(labels_df, on='instancefilename')
print(merge_df.shape)
merge_df.head(40)

(84, 30)


,study,instancefilename,plax_far,plax_plax,plax_laz,psax_az,psax_mv,psax_pap,a2c_lvocc_s,a2c_laocc,...,psax_avz,suprasternal,subcostal,plax_lac,psax_apex,view_pred,sopinstanceuid,instanceidk,view,studyidk
0,90353,E8R9KEOU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00020000001,...,0.0,0.0,0.0,0.0,0.0,a4c_laocc,1.2.840.113663.1500.1.402357036.3.12.20161116....,1976546,a4c,90353
1,91046,EE9Q02L9,0.030699998,0.0006,0.0007000001,0.00040000002,0.0174,0.30920002,0.0006,0.0172,...,0.0007999999,0.017,0.1007,0.015700001,0.1911,psax_pap,1.2.840.113663.1500.1.406857159.3.1.20161128.1...,1994163,plax,91046
2,124700,D2I30O65,0.0,0.0,0.0,0.0,0.0,0.000100000005,0.0,0.0,...,0.0,0.0,0.0,0.00020000001,0.0,other,1.2.840.113663.1500.1.365900332.3.2.20181024.1...,2947248,plax,124700
3,76235,2O353AET,0.0003,0.00080000004,0.0003,0.0025,0.0003,0.000100000005,0.0084,0.0048,...,0.0007999999,0.0012,0.0006,0.0017000001,0.0009,a2c,1.2.840.113663.1500.1.406857159.3.13.20160209....,1547148,a4c,76235
4,101437,7WFYBN1V,0.0,0.0,0.0,0.0,0.0,0.0,0.0072,0.18099998,...,0.0,0.0,0.0,0.00080000004,0.000100000005,other,1.2.840.113663.1500.1.406830382.3.14.20170619....,2285191,a2c,101437
5,114389,43JQTQL9,0.00080000004,0.0,0.0019000001,0.0,0.000100000005,0.0053000003,0.0,0.00040000002,...,0.0,0.0015000001,0.0003,0.0036,0.00080000004,other,1.2.840.113663.1500.1.365900332.3.1.20180321.9...,2651809,plax,114389
6,109593,FGEKD9BU,0.0009,0.0104,0.9202,0.034499995,0.0003,0.0,0.0,0.0,...,0.00090000004,0.0,0.0,0.0051999995,0.0,plax_laz,1.2.840.113663.1500.1.402357036.3.1.20171211.1...,2509083,plax,109593
7,89558,E2BQI0C8,0.0,0.0,0.0,0.0,0.0,0.0,0.000100000005,0.014699998,...,0.000100000005,0.0,0.000100000005,0.0,0.0,a2c,1.2.840.113663.1500.1.406857159.3.23.20161102....,1952899,a2c,89558
8,98709,5FS3GC4P,0.00070000003,0.29520002,0.0077999993,0.0161,0.0005,0.0,0.0,0.0,...,0.00090000004,0.0036999998,0.0011,0.50159997,0.0048,plax_lac,1.2.840.113663.1500.1.406799603.3.2.20170426.1...,2208163,plax,98709
9,100145,6OCAGZHU,0.00080000004,0.0177,0.0106,0.0148,0.00020000001,0.0,0.0,0.0,...,0.0024,0.000100000005,0.0,0.70640004,0.0,plax_lac,1.2.840.113663.1500.1.406799603.3.5.20170523.1...,2247518,plax,100145


In [70]:
# reorder columns
cols = merge_df.columns.tolist()
cols_new = cols[0:2] + list([cols[-2]]) + cols[2:-3] + list([cols[-1]])
#print(cols_new)
merge_df = merge_df[cols_new]

merge_df.rename(index=str, columns={"view": "view_true"}, inplace=True)

merge_df.head()

,study,instancefilename,view_true,plax_far,plax_plax,plax_laz,psax_az,psax_mv,psax_pap,a2c_lvocc_s,...,a5c,other,rvinf,psax_avz,suprasternal,subcostal,plax_lac,psax_apex,sopinstanceuid,studyidk
0,90353,E8R9KEOU,a4c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.2.840.113663.1500.1.402357036.3.12.20161116....,90353
1,91046,EE9Q02L9,plax,0.030699998,0.0006,0.0007000001,0.00040000002,0.0174,0.30920002,0.0006,...,0.0,0.22569999,0.0492,0.0007999999,0.017,0.1007,0.015700001,0.1911,1.2.840.113663.1500.1.406857159.3.1.20161128.1...,91046
2,124700,D2I30O65,plax,0.0,0.0,0.0,0.0,0.0,0.000100000005,0.0,...,0.0,0.9992,0.0,0.0,0.0,0.0,0.00020000001,0.0,1.2.840.113663.1500.1.365900332.3.2.20181024.1...,124700
3,76235,2O353AET,a4c,0.0003,0.00080000004,0.0003,0.0025,0.0003,0.000100000005,0.0084,...,0.0144,0.0313,0.0061,0.0007999999,0.0012,0.0006,0.0017000001,0.0009,1.2.840.113663.1500.1.406857159.3.13.20160209....,76235
4,101437,7WFYBN1V,a2c,0.0,0.0,0.0,0.0,0.0,0.0,0.0072,...,0.0003,0.60800004,0.00020000001,0.0,0.0,0.0,0.00080000004,0.000100000005,1.2.840.113663.1500.1.406830382.3.14.20170619....,101437


134
(84, 28)


ValueError: Length of values does not match length of index

In [82]:
# For each row, get column with highest probability

cols_probs = merge_df.columns.tolist()[3:-2]

df_probs = merge_df[cols_probs]
df_probs['max_prob'] = df_probs.max(axis=1)

df_probs.head()

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,plax_far,plax_plax,plax_laz,psax_az,psax_mv,psax_pap,a2c_lvocc_s,a2c_laocc,a2c,a3c_lvocc_s,...,a4c,a5c,other,rvinf,psax_avz,suprasternal,subcostal,plax_lac,psax_apex,max_prob
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00020000001,0.0,0.0,...,0.0018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9973
1,0.030699998,0.0006,0.0007000001,0.00040000002,0.0174,0.30920002,0.0006,0.0172,0.0006,0.0056000003,...,0.000100000005,0.0,0.22569999,0.0492,0.0007999999,0.017,0.1007,0.015700001,0.1911,0.3092
2,0.0,0.0,0.0,0.0,0.0,0.000100000005,0.0,0.0,0.0,0.0,...,0.0,0.0,0.9992,0.0,0.0,0.0,0.0,0.00020000001,0.0,0.9992
3,0.0003,0.00080000004,0.0003,0.0025,0.0003,0.000100000005,0.0084,0.0048,0.49709997,0.00070000003,...,0.4112,0.0144,0.0313,0.0061,0.0007999999,0.0012,0.0006,0.0017000001,0.0009,0.4971
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0072,0.18099998,0.19860001,0.0,...,0.0003,0.0003,0.60800004,0.00020000001,0.0,0.0,0.0,0.00080000004,0.000100000005,0.6080


In [84]:
print(cols_probs)

['plax_far', 'plax_plax', 'plax_laz', 'psax_az', 'psax_mv', 'psax_pap', 'a2c_lvocc_s', 'a2c_laocc', 'a2c', 'a3c_lvocc_s', 'a3c_laocc', 'a3c', 'a4c_lvocc_s', 'a4c_laocc', 'a4c', 'a5c', 'other', 'rvinf', 'psax_avz', 'suprasternal', 'subcostal', 'plax_lac', 'psax_apex']


In [92]:
for col in cols_probs[0:3]:
#     print(df_probs[col])
    if (df_probs[col].any() == df_probs['max_prob']).any():
        print(col)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)
